In [ ]:
import requests
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import undetected_chromedriver as uc
import locale

In [ ]:

months = {
    "ene": "Jan", "feb": "Feb", "mar": "Mar", "abr": "Apr",
    "may": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug",
    "sept": "Sep", "oct": "Oct", "nov": "Nov", "dic": "Dec"
}

locale.setlocale(locale.LC_TIME, 'C')

json_pages_info = {"pages": []}

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")

driver = uc.Chrome(options=options)

pages = 10

for page in range(0, pages):

	url = f'https://www.google.com/search?q=%22Estallido+social%22+site%3Aelpais.com&tbs=cdr:1,cd_min:11/15/2019,cd_max:12/17/2023&start={page * 10}'

	driver.get(url)

	time.sleep(2)

	articles_section = driver.find_element(By.CLASS_NAME, "dURPMd")
	articles = articles_section.find_elements(By.CLASS_NAME, "MjjYud")

	for article in articles:
		print("revisando artículo...")
		try: 
			originalDate = article.find_element(By.CLASS_NAME, "YrbPuc").find_element(By.TAG_NAME, "span").text
			for es, en in months.items():
				if es in originalDate:
						originalDate = originalDate.replace(es, en)
						break
				
			date_epoch = int(time.mktime(time.strptime(originalDate, "%d %b %Y")))

			if (date_epoch < 1573786800 or date_epoch > 1702782000):
				print("La noticia no corresponde a la fecha solicitada")
				continue

			# check if the article is from El Mostrador
			if (not ("https://elpais.com" in article.text)):
				print("El artículo no es de El Mostrador")
				continue

			try:
				title = article.find_element(By.TAG_NAME, "h3").text
				description = article.find_element(By.CLASS_NAME, "kb0PBd ").find_elements(By.TAG_NAME, "span")[1].text
				link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

				link_info = {
					"newscast" : "El País",
					"title": title,
					"description": description,
					"category": "The site does not provide a category",
					"date": originalDate,
					"image_link": "not found initially",
					"author": "not found initially",
					"link": link,
				}

				print("información incluida!")
				json_pages_info["pages"].append(link_info)

			except Exception as e:
				print(f"Error al extraer información del artículo: {e}")
				continue

		except Exception as e:
			print(f"Error al procesar el artículo: {e}")
			continue

print("Almacenando información en el archivo JSON...")
with open(f"../archive/temp/el_pais_pages_08072025.json", 'w', encoding='utf-8') as file:
	json.dump(json_pages_info, file, ensure_ascii=False, indent=4)


with open(f"output.html", "w", encoding="utf-8") as f:
	f.write(driver.page_source)

driver.quit()

In [ ]:
import json

el_pais_data01 = {}
el_pais_data02 = {}

clean_data = {
	"pages": []
}

with open("../archive/temp/el_pais_pages_with_content_27_05_25.json", 'r', encoding='utf-8') as file:
	el_pais_data01 = json.load(file)

with open("../archive/temp/el_pais_pages_08072025_raw.json", 'r', encoding='utf-8') as file:
	el_pais_data02 = json.load(file)

coincidence = 0

all_data = el_pais_data01["pages"] + el_pais_data02["pages"]
all_links = [page["link"] for page in all_data]

clean_data = {
	"pages": []
}

clean_data_links = []

print("el pais 01 " + str(len(el_pais_data01["pages"])))
print("el pais 02 " + str(len(el_pais_data02["pages"])))
print("all data " + str(len(all_data)))

coincidence_boolean = False

for page in all_data:
	for link in clean_data_links:
		if page["link"] in link:
			print(f"Coincidencia encontrada: {page['link']} con link {link}")
			coincidence += 1
			coincidence_boolean = True
			break
	if not coincidence_boolean:
		print(f"Agregando página: {page['link']}")
		clean_data["pages"].append(page)
		clean_data_links.append(page["link"])
	coincidence_boolean = False


print(f"Total coincidences: {coincidence}")
print(f"Total unique links: {len(clean_data['pages'])}")

with open(f"../archive/temp/el_pais_cleanData_080725.json", 'w', encoding='utf-8') as file:
	json.dump(clean_data, file, ensure_ascii=False, indent=4)
